In [12]:
from lore_sa.dataset import TabularDataset

In [13]:
dataset = TabularDataset.from_csv('iris.csv', class_name = "variety")
dataset.df.dropna(inplace = True)

In [14]:
dataset.df.keys()

Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width',
       'variety'],
      dtype='object')

In [15]:
dataset.update_descriptor()

In [16]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from lore_sa.bbox import sklearn_classifier_bbox


def train_model(dataset: TabularDataset):
    numeric_indices = [v['index'] for v in dataset.descriptor['numeric'].values()]
    categorical_indices = [v['index'] for v in dataset.descriptor['categorical'].values()]
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_indices),
            ('cat', OrdinalEncoder(), categorical_indices)
        ]
    )
    model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, random_state=42))
    
    X_train, X_test, y_train, y_test = train_test_split(dataset.df.loc[:, 'sepal.length':'petal.width'].values, dataset.df['variety'].values,
                test_size=0.3, random_state=42, stratify=dataset.df['variety'].values)
    model.fit(X_train, y_train)
    
    return sklearn_classifier_bbox.sklearnBBox(model)

In [17]:
bbox = train_model(dataset)

In [18]:
from lore_sa.lore import TabularGeneticGeneratorLore

tabularLore = TabularGeneticGeneratorLore(bbox, dataset)

In [19]:
out = []
for k in dataset.descriptor.keys():
    if k != 'target':
        out.extend(list(dataset.descriptor[k].keys()))
out, dataset.descriptor.keys()

(['sepal.length', 'sepal.width', 'petal.length', 'petal.width'],
 dict_keys(['numeric', 'categorical', 'ordinal', 'target']))

In [20]:
dataset.descriptor["target"]

{'variety': {'index': 4,
  'distinct_values': ['Setosa', 'Versicolor', 'Virginica'],
  'count': {'Setosa': 50, 'Versicolor': 50, 'Virginica': 50}}}

In [21]:
from lore_sa.webapp import Webapp
webapp = Webapp()

In [22]:
webapp.launch_demo()
# webapp.interactive_explanation(tabularLore.bbox, tabularLore.dataset, 'variety')

Launching LORE_sa Demo Application
Starting API server on 0.0.0.0:8000


INFO:     Started server process [12892]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


API server is ready at http://localhost:8000
NPM version detected: 10.9.3
Dependencies already installed, skipping npm install
Application started successfully!
API: http://localhost:8000
Client: http://localhost:8080
Opening http://localhost:8080 in your default browser...
Browser opened successfully!
